In [1]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

#https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Using TensorFlow backend.


In [2]:
IMG_PATH = "data/"
print(os.listdir(IMG_PATH))

['test', 'train', 'val']


In [3]:
train_path = IMG_PATH + "train/"
validation_path = IMG_PATH + "val/"

In [4]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")

In [5]:
number_of_imgs(train_path)
number_of_imgs(validation_path)

data/train/
NORMAL has 1326 imgs
PNEUMONIA has 3860 imgs
data/val/
non_pneumonia has 35 imgs
pneumonia has 35 imgs


In [6]:
img_width, img_height = 224, 224
nb_train_samples = 5218
nb_validation_samples = 75
epochs = 10
batch_size = 35
checkpoint_filepath = "checkpoint/model_val_loss-{val_loss:.2f}.h5"

In [7]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

W0827 12:55:38.786608 139938689468160 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 12:55:38.798352 139938689468160 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 12:55:38.800515 139938689468160 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 12:55:38.820404 139938689468160 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf

In [12]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [13]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())                                    
model.add(Dense(64))                                    
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))                                     
model.add(Activation('sigmoid'))            

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                1605696   
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total para

In [15]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    rescale=1. / 255,                   #Scaling RGB values from "0-255" to "0-1" 
    zoom_range=0.2,                 #Random zoom between 0 and 0.2
    horizontal_flip=True) #Random horizontal flip
               


test_datagen = ImageDataGenerator(rescale=1. / 255) #Scaling RGB values from "0-255" to "0-1". 


# Generates batches of image data for training

train_generator = train_datagen.flow_from_directory(
    train_path,                                       #Train data directory
    target_size=(img_width, img_height),                  #Image size
    batch_size=batch_size,
    class_mode="binary")                                  #Amount of classes, binary if 2 classes, else categorical


validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")


# Lowering learning rate if no progress is made.

reduce_learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    min_lr=0.00001,
    verbose=1)


# Defining checkpoint that saves the best model or model"s weights based on the value that we are monitoring.

checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    #save_weights_only=True,
    mode="min")


# Defining callbacks which includes checkpoint and reducing learning rate on plateau.

callbacks = [checkpoint]

# Training CNN

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,         
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks)

Found 5184 images belonging to 2 classes.
Found 68 images belonging to 2 classes.
Epoch 1/10
149/149 [==============================] - 103s 691ms/step - loss: 0.2745 - acc: 0.8823 - val_loss: 0.3540 - val_acc: 0.8382

Epoch 00001: val_loss improved from inf to 0.35405, saving model to checkpoint/model_val_loss-0.35.h5
Epoch 2/10
149/149 [==============================] - 97s 651ms/step - loss: 0.1663 - acc: 0.9373 - val_loss: 0.3059 - val_acc: 0.8529

Epoch 00002: val_loss improved from 0.35405 to 0.30592, saving model to checkpoint/model_val_loss-0.31.h5
Epoch 3/10
149/149 [==============================] - 98s 659ms/step - loss: 0.1364 - acc: 0.9494 - val_loss: 0.1076 - val_acc: 0.9118

Epoch 00003: val_loss improved from 0.30592 to 0.10760, saving model to checkpoint/model_val_loss-0.11.h5
Epoch 4/10
149/149 [==============================] - 100s 672ms/step - loss: 0.1125 - acc: 0.9580 - val_loss: 0.1069 - val_acc: 0.9559

Epoch 00004: val_loss improved from 0.10760 to 0.10694, sa